In [1]:
# Statistical Machine Translation System
# English to Cantonese

# IBM Model 1 for Word Translation Task
# Word Alignment based on Relative Positions
# Bi-gram Language Modelling with Laplace Smoothing and Backoff

In [2]:
!pip install nltk
!pip install -q tensorflow

In [3]:
import pickle
import tensorflow as tf

## Input Data

In [4]:
import pandas as pd
import glob
import os

all_csv = glob.glob(os.getcwd() + "/train.csv")  

df_test = pd.read_csv(os.getcwd() + "/test.csv", sep='\t', encoding='utf-8')  

df_from_each_file = (pd.read_csv(f, sep='\t', encoding='utf-8') for f in all_csv)
df = pd.concat(df_from_each_file)

# Check for null
df[df['yue'].isnull()]
df = df.dropna()
df_test = df_test.dropna()

YueChar = True
# Delete spaces between n-gram in Cantonese
# Perform Character based tokenization in Cantonese
if YueChar:
    df['yue'] = df['yue'].str.replace(r' ', '')
    df_test['yue'] = df_test['yue'].str.replace(r' ', '')

df.head()

,yue,eng
0,唔準掂同食醃菜唔準坐梳化或者屋企人嘅床上每次經期完咗要洗床單就算床單無汚糟到他們認為我唔純潔...,I was not allowed to touch or eat pickles I wa...
1,佢地扮怪面嚇人,They were making scary faces
2,唔會搞到我哋變得邪惡女人型變成自由黨人咩,turn us into godless sissy liberals
3,呢個模式可以清晰令您瞭解佢哋,So it spells those out in very clean terms
4,幾多萬億掌聲,How many trillions Applause


In [5]:
import regex as re
def spliteKeyWord(str):
    regex = r"[\u4e00-\ufaff]|[0-9]+|[a-zA-Z]+\'*[a-z]*"
    matches = re.findall(regex, str, re.UNICODE)
    return ' '.join(matches)

if YueChar:
    df['yue'] = df['yue'].apply(lambda x: spliteKeyWord(x))
    df_test['yue'] = df_test['yue'].apply(lambda x: spliteKeyWord(x))
df_test.head()

,yue,eng
0,我 相 信 主,I believe the almighty
1,好 耐 以 嚟 有 發 展 紊 亂 嘅 小 朋 友,For too long now children with developmental d...
2,一 般 會 遇 到 兩 種 反 應,I have two kinds of reactions
3,但 再 諗 下 嗰 位 官 員 未 必 係 唯 一 睇 小 女 性 嘅 人 呢 種 偏 見 ...,But think about this The IMF official is hardl...
4,佢 將 呢 個 病 毒 傳 畀 BB,She passes that virus on to baby


In [6]:
MAX_LENGTH = 40
inp_lang = 'eng'
tar_lang = 'yue'

def df_filter(df):
    return df[
        df['yue'].apply(lambda x: len(x.split())<MAX_LENGTH) &
        df['eng'].apply(lambda x: len(x.split())<MAX_LENGTH) 
    ]
df = df_filter(df)
df_test = df_filter(df_test)

inp = [i.split() for i in df[inp_lang].to_list()]
tar = [i.split() for i in df[tar_lang].to_list()]
inp_test = [i.split() for i in df_test[inp_lang].to_list()]
tar_test = [i.split() for i in df_test[tar_lang].to_list()]
inp[:3], tar[:3]

([['They', 'were', 'making', 'scary', 'faces'],
  ['turn', 'us', 'into', 'godless', 'sissy', 'liberals'],
  ['So', 'it', 'spells', 'those', 'out', 'in', 'very', 'clean', 'terms']],
 [['佢', '地', '扮', '怪', '面', '嚇', '人'],
  ['唔',
   '會',
   '搞',
   '到',
   '我',
   '哋',
   '變',
   '得',
   '邪',
   '惡',
   '女',
   '人',
   '型',
   '變',
   '成',
   '自',
   '由',
   '黨',
   '人',
   '咩'],
  ['呢', '個', '模', '式', '可', '以', '清', '晰', '令', '您', '瞭', '解', '佢', '哋']])

In [7]:
train_size = len(inp)
test_size = len(inp_test)
train_size, test_size

(6851, 1696)

### Vocabulary

In [8]:
inp_words = {}
tar_words = {}

for sentence in inp:
    for word in sentence:
        if word in inp_words:
            inp_words[word] += 1
        else:
            inp_words[word] = 1
            
for sentence in tar:
    for word in sentence:
        if word in tar_words:
            tar_words[word] += 1
        else:
            tar_words[word] = 1
                    
inp_vocab = len(inp_words)
tar_vocab = len(tar_words)
print("Number of Unique Words:")
print(inp_lang, ':', str(inp_vocab))
print(tar_lang, ':', str(tar_vocab))

Number of Unique Words:
eng : 6219
yue : 2705


# Training and checkpointing

In [9]:
# creating the 't'
t = {}
# usage: t[('inp_word', 'tar_word')] = probability of inp_Word given tar_word
uniform = 1 / (inp_vocab * tar_vocab)

hyperparameter

In [10]:
max_iters = 128
word_factor_max = 2

fine_tune = 1
has_converged = False

In [11]:
run_id = f"SMT-2_EngYue"
log_dir = os.path.join(os.path.join(os.getcwd(), 'log'), run_id)
%load_ext tensorboard
%tensorboard --logdir {log_dir}

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
2021-04-16 14:52:16.435526: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-04-16 14:52:16.435695: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--purge_orphaned_data BOOL] [--db URI] [--db_import]
                   [--inspect] [--version_tb] [--tag TAG] [--event_file PATH]
                   [--path_prefix PATH] [--window_title TEXT]
                   [--max_reload_threads COUNT] [--reload_interval SECONDS]
                   [--reload_task TYPE] [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data

In [12]:
import time
n_iters = 0

while n_iters < max_iters and has_converged == False:
    start = time.time()
    has_converged = True
    max_change = -1

    n_iters += 1
    count = {}
    total = {}
    for index in range(train_size):
        s_total = {}
        for inp_word in inp[index]:
            s_total[inp_word] = 0
            for tar_word in tar[index]:
                if (inp_word, tar_word) not in t:
                    t[(inp_word, tar_word)] = uniform
                s_total[inp_word] += t[(inp_word, tar_word)]

        for inp_word in inp[index]:
            for tar_word in tar[index]:
                if (inp_word, tar_word) not in count:
                    count[(inp_word, tar_word)] = 0
                count[(inp_word, tar_word)] += (t[(inp_word, tar_word)] / s_total[inp_word])

                if tar_word not in total:
                    total[tar_word] = 0
                total[tar_word] += (t[(inp_word, tar_word)] / s_total[inp_word])

    # estimating the probabilities

    if fine_tune == 0:
      updated = {}
      # train for all valid word pairs s.t count(inp_word, tar_word) > 0
      for index in range(train_size):
          for tar_word in tar[index]:
              for inp_word in inp[index]:
                  if (inp_word, tar_word) in updated:
                      continue
                  updated[(inp_word, tar_word)] = 1
                  if abs(t[(inp_word, tar_word)] - count[(inp_word, tar_word)] / total[tar_word]) > 0.01:
                      has_converged = False
                      max_change = max(max_change, abs(t[(inp_word, tar_word)] - count[(inp_word, tar_word)] / total[tar_word]))
                  t[(inp_word, tar_word)] = count[(inp_word, tar_word)] / total[tar_word]

    elif fine_tune == 1:
      # train it only for 1000 most frequent words in English and Cantonese
      n_tar_words = 0
      updates = 0

      for tar_word_tuples in sorted(tar_words.items(), key = lambda k:(k[1], k[0]), reverse = True):
          tar_word = tar_word_tuples[0]
          n_tar_words += 1
          if n_tar_words > len(tar_words)/word_factor_max:
              break
          n_inp_words = 0
          for inp_word_tuples in sorted(inp_words.items(), key = lambda k:(k[1], k[0]), reverse = True):
              inp_word = inp_word_tuples[0]
              n_inp_words += 1
              if n_inp_words > len(inp_words)/word_factor_max:
                  break
              if (inp_word, tar_word) not in count or tar_word not in total:
                  continue
                  # assume in this case: t[(inp_word, tar_word)] = uniform
              else:
                  if abs(t[(inp_word, tar_word)] - count[(inp_word, tar_word)] / total[tar_word]) > 0.005:
                      has_converged = False
                      max_change = max(max_change, abs(t[(inp_word, tar_word)] - count[(inp_word, tar_word)] / total[tar_word]))
                  t[(inp_word, tar_word)] = count[(inp_word, tar_word)] / total[tar_word]
                
    summary_writer = tf.summary.create_file_writer(log_dir)
    with summary_writer.as_default():
        tf.summary.scalar("Change", max_change, step=n_iters)

    print("Iteration " + str(n_iters) + " Completed, Maximum Change: " + str(max_change) + '\nTime: {} secs'.format(time.time() - start))


Iteration 1 Completed, Maximum Change: 0.8167420220033955
Time: 20.438411474227905 secs
Iteration 2 Completed, Maximum Change: 0.3456606319708041
Time: 17.682696104049683 secs
Iteration 3 Completed, Maximum Change: 0.18087302712841014
Time: 17.102671146392822 secs
Iteration 4 Completed, Maximum Change: 0.10117600644019553
Time: 20.732101678848267 secs
Iteration 5 Completed, Maximum Change: 0.059758809816697234
Time: 19.197176456451416 secs
Iteration 6 Completed, Maximum Change: 0.03809698662111716
Time: 21.44188356399536 secs
Iteration 7 Completed, Maximum Change: 0.027846464415475086
Time: 20.426174879074097 secs
Iteration 8 Completed, Maximum Change: 0.023560604238536953
Time: 18.342814207077026 secs
Iteration 9 Completed, Maximum Change: 0.019136175287770896
Time: 20.06472897529602 secs
Iteration 10 Completed, Maximum Change: 0.015195646884349101
Time: 19.027939319610596 secs
Iteration 11 Completed, Maximum Change: 0.012661626124247427
Time: 19.509644269943237 secs
Iteration 12 Comp

In [13]:
# displaying the most confident translation pairs
limit = 40
for element in sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True):
  print(element)
  limit -= 1
  if limit <= 0:
    break

(('Applause', '掌'), 0.993822137807804)
(('Guitar', '吉'), 0.9602192290684652)
(('Laughter', '笑'), 0.9443585196973706)
(('or', '或'), 0.8445960533949959)
(('play', '玩'), 0.7203063887893961)
(('cancer', '癌'), 0.719850020241164)
(('I', '我'), 0.669443309056773)
(('because', '因'), 0.6601850305401723)
(('you', '你'), 0.6550438780164455)
(('and', '和'), 0.6464206172605996)
(('Thank', '謝'), 0.6028772693167712)
(('information', '息'), 0.5999382245120436)
(('serious', '嚴'), 0.5943731086801906)
(('like', '似'), 0.5868679633177576)
(('carousel', '迴'), 0.5848280179345284)
(('know', '知'), 0.5707580790486463)
(('and', '及'), 0.5662515494161114)
(('two', '兩'), 0.5602898592921483)
(('apartment', '寓'), 0.5521367285349217)
(('Bible', '聖'), 0.5420702303674806)
(('Laughter', '聲'), 0.53644230213268)
(('So', '所'), 0.5320624787227191)
(('fell', '跌'), 0.5301091595256303)
(('and', '同'), 0.5298744938454806)
(('love', '愛'), 0.5280797775404907)
(('revolution', '革'), 0.5099479899415553)
(('Applause', '鼓'), 0.5049600322047

In [14]:
# saving the translation model
file = open("translation_model.pkl","wb")
pickle.dump(t, file)
file.close()

In [15]:
# using the model trained until convergence
# to use a saved model
model_name = "translation_model.pkl"
pickle_in = open(model_name,"rb")
t = pickle.load(pickle_in)

In [16]:
I = {}
for index in range(train_size):
    for inp_id in range(len(inp[index])):
        length = len(inp[index])
        if length not in I:
            I[length] = {} # maps the positional difference to a tuple: (sum of t's, count)
        for tar_id in range(len(tar[index])):
            if (tar_id - inp_id) not in I[length]:
                I[length][(tar_id - inp_id)] = [t[(inp[index][inp_id], tar[index][tar_id])], 1]
            else:
                I[length][(tar_id - inp_id)][0] += t[(inp[index][inp_id], tar[index][tar_id])]
                I[length][(tar_id - inp_id)][1] += 1

In [17]:
# viewing the available sentence lengths encountered during training
sentence_lengths = []
for key in I.keys():
    if key not in sentence_lengths:
        sentence_lengths.append(key)
sentence_lengths.sort()
print(sentence_lengths)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 37, 38]


In [18]:
# computing the alignment probabilities
# p[I][tar_id - inp_id] = p(i | i', I)

p = {}
for key in I.keys():
    p[key] = {}
    sum_val = 0
    for diff in I[key].keys():
        p[key][diff] = I[key][diff][0] / I[key][diff][1]
        sum_val += p[key][diff]
    for diff in p[key].keys():
        p[key][diff] /= sum_val

In [19]:
for index in range(train_size):
    length_inp = len(inp[index])
    length_tar = len(tar[index])
    if length_tar - length_inp > 10 and length_inp == 1:
        print("Length of English Sentence:", str(length_inp))
        print("Length of Cantonese Sentence:", str(length_tar))
        
# there exists an English sentence with one token s.t the Cantonese translation contains 19 tokens

Length of English Sentence: 1
Length of Cantonese Sentence: 14
Length of English Sentence: 1
Length of Cantonese Sentence: 13


In [20]:
# computing initial transitions
init = {}
for length in p:
    max_prob = -1
    max_jump = 0
    for key in p[length].keys():
        if p[length][key] > max_prob:
            max_prob = p[length][key]
            max_jump = key
    init[length] = max_jump

In [21]:
# computing the transition probabilities for Cantonese
bigrams = {}
unigrams = {}

# training on the train_set
def model(dataset_size, dataset_name):
    global bigrams
    global unigrams
    for index in range(dataset_size):
        token_A = ''
        for tar_token in tar[index]:
            if tar_token not in unigrams:
                unigrams[tar_token] = 1
            else:
                unigrams[tar_token] += 1
            
            token_B = tar_token
            if (token_A, token_B) not in bigrams:
                bigrams[(token_A, token_B)] = 1
            else:
                bigrams[(token_A, token_B)] += 1
            token_A = token_B

model(train_size, 'tar_train')

bigram_count = len(bigrams)
unigram_count = len(unigrams)
print("Number of Unique Bigrams:", bigram_count)
print("Number of Unique Unigrams:", unigram_count)

Number of Unique Bigrams: 27343
Number of Unique Unigrams: 2705


In [22]:
from itertools import permutations

computed_sentences = []
total_BLEU = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 7: 0}
null_BLEU_count = 0

sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)

def find_translation(inp_token):
    for element in sorted_t:
        if element[0][0].lower() == inp_token:
            return element[0][1]
    return ""

def get_prob(seq):
    # bigram language model with laplace smoothing and backoff
    if len(seq) < 2 or len(seq) > 10:
        return 1
    score = 0
    token_A = ''
    for tar_token in seq:
        token_B = tar_token
        if (token_A, token_B) not in bigrams:
            if token_B not in unigrams:
                continue
            else:
                score += unigrams[token_B] / unigram_count
        else:
            base_token_count = 0
            if token_A in unigrams:
                base_token_count = unigrams[token_A]
            score += (bigrams[(token_A, token_B)] + 1) / (base_token_count + unigram_count)
        token_A = token_B
    return score

In [23]:
TotalTime = time.time()
start = time.time()

ori = []
ref = []
can = []
num_test = 0

for index in range(test_size):

    translated_words = []
    for inp_token in inp_test[index]:
        translation = find_translation(inp_token)
        if translation != "":
            translated_words.append(translation)
    perm = permutations(translated_words)
    best_seq = translated_words
    best_prob = -1

    if len(inp_test[index]) < 10 and len(inp_test[index]) > 2:
        for seq in perm:            
            prob = get_prob(seq)
            if prob > best_prob:
                best_prob = prob
                best_seq = seq
            
    ori.append(inp_test[index])
    ref.append(tar_test[index])
    can.append(best_seq)
    num_test += 1
    
    if num_test % 50 == 0:
        print(f'Progress: {num_test} / {test_size} in {time.time()-start} s')
        start = time.time()
        
print(f'\nToTal: {num_test} / {test_size} in {time.time()-TotalTime} s')

Progress: 50 / 1696 in 18.91235852241516 s
Progress: 100 / 1696 in 15.70876145362854 s
Progress: 150 / 1696 in 23.508580923080444 s
Progress: 200 / 1696 in 15.550612211227417 s
Progress: 250 / 1696 in 10.111546754837036 s
Progress: 300 / 1696 in 11.902549266815186 s
Progress: 350 / 1696 in 16.584648609161377 s
Progress: 400 / 1696 in 10.494550704956055 s
Progress: 450 / 1696 in 13.929544687271118 s
Progress: 500 / 1696 in 14.463541269302368 s
Progress: 550 / 1696 in 11.39703369140625 s
Progress: 600 / 1696 in 12.185068130493164 s
Progress: 650 / 1696 in 15.048619508743286 s
Progress: 700 / 1696 in 13.103516578674316 s
Progress: 750 / 1696 in 14.15402889251709 s
Progress: 800 / 1696 in 20.21412706375122 s
Progress: 850 / 1696 in 16.793059587478638 s
Progress: 900 / 1696 in 14.0885169506073 s
Progress: 950 / 1696 in 16.252156496047974 s
Progress: 1000 / 1696 in 15.219541549682617 s
Progress: 1050 / 1696 in 14.501519680023193 s
Progress: 1100 / 1696 in 8.862546682357788 s
Progress: 1150 /

In [24]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction

import random
ran = random.randint(1, len(ref))
print(ori[ran])
print(ref[ran])
print(can[ran])

smoothie = SmoothingFunction().method4
print('BLEU:', sentence_bleu(ref[ran], can[ran], smoothing_function=smoothie)*100)

['because', 'when', 'the', 'self', 'is', 'suspended']
['因', '為', '當', '我', '的', '自', '我', '被', '暫', '停', '後']
('界', '因', '當', '自', '是', '暫')
BLEU: 103.96602697150126


In [25]:
# Sentence-based and average score
score = 0
# for i in range(len(ref)):
#     r = ref[i]
#     c = can[i]
#     score += sentence_bleu(r, c, smoothing_function=smoothie)*100
# print('BLEU-s:', score/len(ref))

# Corpus based, summing all nominator and denominator before division
# r = [[r.split()] for r in ref]
# c = [c.split() for c in can]
score = corpus_bleu(ref, can, smoothing_function=smoothie)*100
print('BLEU-c:', score)

BLEU-c: 0.18080344988234295
